# Plant image classification model
This notebook will cover the steps of training and infering plant type from an image.

These are the steps in this notebook:
1. Setup workspace

The first step to work clean, is to create a clean workspace. <br/>
And download all the necessary files from pypi

In [1]:
import os
from enum import StrEnum

class Paths(StrEnum):
    DATA = 'data'
    OUTPUT = 'output'

for folder_name in Paths:
    os.makedirs(folder_name, exist_ok=True)

After we have created the workspace directories. We need to get some data. For that we will download the flower photos dataset from the given URL link (or you can download it manually from the tensorflow site).

In [3]:
import requests
from tqdm import tqdm

def download(url: str, filename: str) -> None:
    """
    Downloading a file and showing some indecation about the download using `tqdm`

    Args:
        `url: str` - The URL to download from the file
        `filename: str` - The path to save the downloaded file in the machine
    """
    with open(filename, 'wb') as f:
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            total = int(r.headers.get('content-length', 0))

            # tqdm has many interesting parameters. Feel free to experiment!
            tqdm_params = {
                'desc': url,
                'total': total,
                'miniters': 1,
                'unit': 'B',
                'unit_scale': True,
                'unit_divisor': 1024,
            }
            with tqdm.tqdm(**tqdm_params) as pb:
                for chunk in r.iter_content(chunk_size=8192):
                    pb.update(len(chunk))
                    f.write(chunk)

# DATASET_URL = 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'
# filename = os.path.join(Paths.DATA, 'flower_photos.tgz')
# download(DATASET_URL, filename)

Exctracting the dataset from the compressed file

In [4]:
import tarfile
from tarfile import TarInfo

def extract_file(filename: str, extraction_path: str) -> None:
    """
    Extracting all the contents of tarfile

    Args:
        `filename: str` - The path to the compressed tarfile
        `extraction_path: str` - The path to the target directory which will contain the extracted contents
    """
    with tarfile.open(filename, 'r:gz') as tar:
        total_files: list[TarInfo] = tar.getmembers()

        with tqdm(total=len(total_files), unit='file', desc='Extracting') as progress_bar:
            for member in total_files:
                tar.extract(member, extraction_path)
                progress_bar.update(1)

extract_file(os.path.join(Paths.DATA, 'flower_photos.tgz'), Paths.DATA)

Extracting:   0%|          | 0/3677 [00:00<?, ?file/s]C:\Users\user\AppData\Local\Temp\ipykernel_10300\3578066170.py:17: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extract(member, extraction_path)
Extracting: 100%|██████████| 3677/3677 [00:18<00:00, 201.14file/s]
